In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="D90rVo7ahGyBHqpqtBIH")
project = rf.workspace("indian-institute-of-information-technology-sricity").project("guava-h98xp")
dataset = project.version(6).download("tensorflow")

In [ ]:
!pip install tensorflow
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score
# import tensorflow as tf
import matplotlib.image as mpimg
from sklearn.preprocessing import FunctionTransformer,LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


In [ ]:
def load_data(dir , df):
    # Load images and labels into arrays
    images = []
    labels = []

    for index, row in df.iterrows():
        image_file_path = os.path.join(dir, row['filename'])
        image = cv2.imread(image_file_path)
        if image is not None:
            resized_image = cv2.resize(image, (img_size, img_size))  # Resize to a fixed size
            resized_image = np.array(resized_image) / 255.0
        images.append(resized_image)
        labels.append(row['target'])

    # Convert lists to numpy arrays
    return np.array(images) , np.array(labels)

In [ ]:
img_size = 350

In [ ]:
dir = "/kaggle/working/guava-6/train"
df = pd.read_csv(dir + "/_annotations.csv")
label_encoder = LabelEncoder()
# Encode the 'class' column
df['target'] = label_encoder.fit_transform(df['class'])

train_data, train_labels = load_data(dir , df)

print("Loaded data shape:", train_data.shape)
print("Loaded labels shape:", train_labels.shape)


In [ ]:
dir = "/kaggle/working/guava-6/test"
df = pd.read_csv(dir + "/_annotations.csv")
label_encoder = LabelEncoder()
# Encode the 'class' column
df['target'] = label_encoder.fit_transform(df['class'])

test_data, test_labels = load_data(dir , df)

print("Loaded data shape:", test_data.shape)
print("Loaded labels shape:", test_labels.shape)

In [ ]:
dir = "/kaggle/working/guava-6/valid"
df = pd.read_csv(dir + "/_annotations.csv")
label_encoder = LabelEncoder()
# Encode the 'class' column
df['target'] = label_encoder.fit_transform(df['class'])

valid_data, valid_labels = load_data(dir , df)

print("Loaded data shape:", valid_data.shape)
print("Loaded labels shape:", valid_labels.shape)

Resnet 50


In [ ]:
# Encode train_labels, valid_labels, and test_labels using one-hot encoding
train_labels_encoded = to_categorical(train_labels)
valid_labels_encoded = to_categorical(valid_labels)
test_labels_encoded = to_categorical(test_labels)

In [ ]:
import pandas as pd
from keras.applications import VGG16, InceptionV3, MobileNet, DenseNet121, ResNet50V2
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create and compile the CNN model
def create_and_compile_model_CNN(img_size, num_classes, base_model):
    base_model = base_model(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create a directory to save models
if not os.path.exists('./models'):
    os.makedirs('./models')

models = {
    "VGG16": VGG16,
    "InceptionV3": InceptionV3,
    "MobileNet": MobileNet,
    "DenseNet121": DenseNet121,
    "ResNet50V2": ResNet50V2
}

num_classes = 2
batch_size = 8
epochs = 10
img_size = 350
results = []

# Data loading and preprocessing should be done here (train_data, train_labels_encoded, valid_data, valid_labels_encoded, test_data, test_labels).

for model_name, base_model in models.items():
    print(f"Training {model_name}...")
    model = create_and_compile_model_CNN(img_size, num_classes, base_model)
    model.fit(train_data, train_labels_encoded, batch_size=batch_size, epochs=epochs, validation_data=(valid_data, valid_labels_encoded))
    
    # Save the model
    model.save(f'./models/{model_name}_model.h5')
    
    predictions = model.predict(test_data)
    predicted_labels = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(test_labels, predicted_labels)
    print(f"{model_name} Accuracy: {accuracy:.2f}")
    
    # Generate and save confusion matrix
    cm = confusion_matrix(test_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.savefig(f'./models/{model_name}_confusion_matrix.png')
    
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Predicted Labels": predicted_labels.tolist()
    })

df = pd.DataFrame(results)
# print(df)
df.to_csv("model_results.csv", index=False)

In [ ]:
df